In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# define global parameters
n_estimators = 300
max_depth = 18
min_samples_split = 4
min_samples_leaf = 2
seed = 23

def clean_train_data(file):
    data = pd.read_csv(file)
    init_features = ["Survived", "Pclass", "Sex", "SibSp", "Parch", "Embarked"] # choose relevant features
    data = data[init_features]
    data = pd.get_dummies(data) # one hot encode categorical values
    return data

def get_vals(data):
    """get X and y to use in training and testing model"""
    features = ["Pclass", 
                "SibSp", 
                "Parch", 
                "Sex_female", 
                "Sex_male", 
                "Embarked_C", 
                "Embarked_Q", 
                "Embarked_S"] # update features based on OHE

    X = data[features] # values to learn from 
    y = data["Survived"] # values to predict
    return X, y

def fit_model():
    """fit model to train data then make predictions on test data"""
    rf_model = RandomForestClassifier(n_estimators=n_estimators, 
                                    max_depth=max_depth, 
                                    min_samples_leaf=min_samples_leaf, 
                                    min_samples_split=min_samples_split, 
                                    random_state=seed) # initialize random forest model
    rf_model.fit(X, y) # fit model to training data
    return rf_model

def clean_test_data(file):
    data = pd.read_csv(file)
    init_features = ["PassengerId", "Pclass", "Sex", "SibSp", "Parch", "Embarked"] # choose relevant features
    data = data[init_features]
    data = pd.get_dummies(data) # one hot encode categorical values
    return data

cleaned_data = clean_train_data("/kaggle/input/titanic/train.csv")
X, y = get_vals(cleaned_data)
model = fit_model()
test_data = clean_test_data("/kaggle/input/titanic/test.csv")
test_X = test_data.drop("PassengerId", axis=1)
preds = model.predict(test_X) # get predictions based on test data

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': preds})
output.to_csv('submission.csv', index=False) # write solution out to csv for submission